In [6]:
import psycopg2

# 1- What is the average order price per customer?

In [10]:
# create connection to postgresql 
conn = psycopg2.connect(database = "dwh_db", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")

# create cursor
cursor = conn.cursor()

ans1_query = '''
select FF.cust_id,FF.name,avg(FF.sumprice)
from
(select DC.cust_id,DC.name,F.orderid,sum(F.quantity*F.uprice) AS sumprice  from factorder F
inner join dimcustomer DC
on F.c_id = DC.id
group by DC.cust_id,DC.name, F.orderid) FF
group by FF.cust_id,FF.name
 '''

cursor.execute(ans1_query )
records = cursor.fetchall()

cursor.close()
conn.close()

In [11]:
records

[('CUST-003', 'Smith', Decimal('49.9900000000000000')),
 ('CUST-002', 'Doe', Decimal('192.9700000000000000')),
 ('CUST-004', 'Parker', Decimal('48.9900000000000000')),
 ('CUST-001', 'Jones', Decimal('121.6300000000000000'))]

# 2-1 How many ‘Vacuum cleaners’ were ordered in New York?

In [12]:
# create connection to postgresql 
conn = psycopg2.connect(database = "dwh_db", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")

# create cursor
cursor = conn.cursor()

ans1_query = '''
select DP.product_id,DP.name,DC.city,sum(F.quantity) AS sumquantity  from factorder F
inner join dimproduct DP
on F.p_id = DP.product_id
inner join dimcustomer DC
on F.c_id = DC.id
where DP.name='Vacuum cleaner' and DC.city='New York'
group by DP.product_id,DP.name ,DC.city

 '''

cursor.execute(ans1_query )
records = cursor.fetchall()

cursor.close()
conn.close()

In [13]:
records

[(1, 'Vacuum cleaner', 'New York', 4)]

# 2-2 who bought most of them (Vacuum cleaners) in this city?

In [14]:
# create connection to postgresql 
conn = psycopg2.connect(database = "dwh_db", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")

# create cursor
cursor = conn.cursor()

ans1_query = '''
select 
    FU.product_id,FU.name,FU.city,FU.custname,FU.sumquantity
from 
(select DP.product_id,DP.name,DC.cust_id,DC.name as custname,DC.city,sum(F.quantity) AS sumquantity,
 rank() over (partition by DP.product_id,DP.name,DC.city order by sum(F.quantity) desc ) as recordrank
 from factorder F
inner join dimproduct DP
on F.p_id = DP.product_id
inner join dimcustomer DC
on F.c_id = DC.id
where DP.name='Vacuum cleaner' and DC.city='New York'
group by DP.product_id,DP.name,DC.cust_id,DC.name,DC.city) FU
where FU.recordrank = 1
 '''

cursor.execute(ans1_query )
records = cursor.fetchall()

cursor.close()
conn.close()

In [15]:
records

[(1, 'Vacuum cleaner', 'New York', 'Jones', 4)]

# 3 - What product is the most popular in each city?

In [16]:
# create connection to postgresql 
conn = psycopg2.connect(database = "dwh_db", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")

# create cursor
cursor = conn.cursor()

ans1_query = '''
select
K.city,K.product_id,K.name,K.sumquantity
from
(select DP.product_id,DP.name,DC.city,sum(F.quantity) AS sumquantity,
 rank() over (partition by DC.city order by sum(F.quantity) desc ) as recordrank
 from factorder F
inner join dimproduct DP
on F.p_id = DP.product_id
inner join dimcustomer DC
on F.c_id = DC.id
group by DP.product_id,DP.name,DC.city) K
where K.recordrank=1
 '''

cursor.execute(ans1_query )
records = cursor.fetchall()

cursor.close()
conn.close()

In [17]:
records

[('Atlanta', 1, 'Vacuum cleaner', 3),
 ('Baltimore', 1, 'Vacuum cleaner', 1),
 ('Denver', 1, 'Vacuum cleaner', 1),
 ('New York', 2, 'Cleaner bags', 5)]